In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.preprocessing.sequence import pad_sequences
filepath=[r"B:\project\human\human_chunk_16.csv",r"B:\project\ai\ai_chunk_4.csv"]
dfs = [pd.read_csv(filepath) for filepath in filepath]
df = pd.concat(dfs, ignore_index=True)
print(df.shape)


(30580, 2)


In [2]:
essay=df['text'].tolist()
from transformers import LongformerTokenizer
tokenizer=LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
essay=df['text'].tolist()
tokens=tokenizer(essay,return_tensors='pt',padding=True,truncation=True)

In [3]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
# input_ids_size = tokens['input_ids'].size()
# attention_mask_size = tokens['attention_mask'].size()
# print("Input IDs size:", input_ids_size)
# print("Attention Mask size:", attention_mask_size)
# # Assuming 'tokens' is the result of tokenizer
# unique_tokens = set(token.item() for seq in tokens['input_ids'] for token in seq)
# vocabulary_size = len(unique_tokens)
# print("Vocabulary Size:", vocabulary_size)

In [4]:
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model
xtrain_np=xtrain.numpy()
input_length = None
input_layer = Input(shape=(input_length,))
embedding_layer = Embedding(input_dim=51000, output_dim=100, input_length=input_length)(input_layer)
conv1d_layer = Conv1D(128, 5, activation='relu')(embedding_layer)
maxpool_layer = GlobalMaxPooling1D()(conv1d_layer)
dense_layer = Dense(64, activation='relu')(maxpool_layer)
dropout_layer = Dropout(0.5)(dense_layer)
output_layer = Dense(1, activation='sigmoid')(dropout_layer)
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Assuming xtrain_np and ytrain are NumPy arrays
model.fit(xtrain_np, ytrain, epochs=5, batch_size=256)



Epoch 1/5


96/96 [==============================] - 73s 747ms/step - loss: 0.3549 - accuracy: 0.8638
Epoch 2/5
96/96 [==============================] - 72s 751ms/step - loss: 0.0359 - accuracy: 0.9907
Epoch 3/5
96/96 [==============================] - 73s 755ms/step - loss: 0.0084 - accuracy: 0.9985
Epoch 4/5
96/96 [==============================] - 73s 764ms/step - loss: 0.0030 - accuracy: 0.9997
Epoch 5/5
96/96 [==============================] - 72s 751ms/step - loss: 0.0015 - accuracy: 1.0000


In [5]:
xtest_np=xtest.numpy()
model.evaluate(xtest_np,ytest)

192/192 [==============================] - 4s 20ms/step - loss: 0.0056 - accuracy: 0.9972


[0.005562384147197008, 0.9972203969955444]

In [6]:
model.save("dmodel16.keras")
model.save_weights('dmodel16_weights.keras')

In [7]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
xtrain_np=xtrain.numpy()
xtest_np=xtest.numpy()
xtrain_np16=xtrain_np
xtest_np16=xtest_np
import numpy as np
xtrain_np16 = np.array(xtrain_np16)
xtest_np16=np.array(xtest_np16)
np.savetxt('B:/project/test/xtest_np16.csv', xtest_np16, delimiter=',')
np.savetxt('B:/project/train/xtrain_np16.csv', xtrain_np16, delimiter=',')
ytest.to_csv("B:/project/ytest/ytest16.csv")

In [8]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
ytrain.to_csv("B:/project/ytrain/ytrain16.csv")